In [1]:
 # Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
import os
import matplotlib.pyplot as plt#visualization
from PIL import  Image
%matplotlib inline
import pandas as pd
import seaborn as sns#visualization
import itertools
import warnings
warnings.filterwarnings("ignore")
import io
from pylab import rcParams
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score,recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

In [0]:
df=pd.read_csv("/content/scrapeyt.csv")

In [6]:
df.head()

,Video Id,Title,Description,Category
0,\n Top 5 Amazing Unique Gadgets...,NaN,Top 5 Amazing Unique Gadgets You Must Watch No...,Science and Technology
1,/watch?v=9AMPsDXGAxY,\n VERY SMART TECHNOLOGIES THAT...,For copyright matters please contact us at: co...,Science and Technology
2,/watch?v=JhHMJCUmq28,\n Quantum Computers Explained ...,Where are the limits of human technology? And ...,Science and Technology
3,/watch?v=SSo_EIwHSd4&t=115s,\n How does a blockchain work -...,What is a blockchain and how do they work? I'l...,Science and Technology
4,/watch?v=-y5Z2fmnp-o,\n Top 10 Technologies To Learn...,In this Intellipaat's top 10 technologies to l...,Science and Technology


In [8]:
df.shape

(7245, 4)

In [11]:
print(df.shape)
print(df.describe())
print(df.head())
print(df.isnull().sum())

(7245, 4)
                    Video Id  ... Category
count                   7244  ...     7245
unique                  6978  ...        6
top     /watch?v=tNkZsRW7h2c  ...   Travel
freq                       4  ...     1442

[4 rows x 4 columns]
                                            Video Id  ...                Category
0  \n                Top 5 Amazing Unique Gadgets...  ...  Science and Technology
1                               /watch?v=9AMPsDXGAxY  ...  Science and Technology
2                               /watch?v=JhHMJCUmq28  ...  Science and Technology
3                        /watch?v=SSo_EIwHSd4&t=115s  ...  Science and Technology
4                               /watch?v=-y5Z2fmnp-o  ...  Science and Technology

[5 rows x 4 columns]
Video Id         1
Title            2
Description    252
Category         0
dtype: int64


In [0]:
df=df.dropna(axis=0)

In [14]:
df.shape

(6992, 4)

In [0]:
# Divide data into training and validation subsets to prevent overfitting 
X_train_full, X_valid_full, y_train, y_valid = train_test_split(df, df.Category, train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [0]:
train_feat=[str(i) for i in X_train_full["Description"]]
valid_feat=[str(i) for i in X_valid_full["Description"]]


In [0]:
tf_feat= TfidfVectorizer(analyzer='word',stop_words="english",
                         ngram_range=(1,2))

In [0]:
idf_feat=tf_feat.fit(train_feat)
train_mat=idf_feat.transform(train_feat)
valid_mat=idf_feat.transform(valid_feat)

In [92]:
train_mat

<5593x61950 sparse matrix of type '<class 'numpy.float64'>'
	with 121272 stored elements in Compressed Sparse Row format>

In [93]:
valid_mat

<1399x61950 sparse matrix of type '<class 'numpy.float64'>'
	with 17818 stored elements in Compressed Sparse Row format>

In [0]:
clf=LogisticRegression()
clf.fit(train_mat,y_train)
preds=clf.predict(valid_mat)



In [95]:
print (classification_report(y_true=y_valid,y_pred=preds))

                        precision    recall  f1-score   support

         Art and Music       0.79      0.74      0.76       231
                  Food       0.84      0.80      0.82       163
               History       0.85      0.65      0.73       205
         Manufacturing       0.89      0.78      0.83       252
Science and Technology       0.68      0.82      0.75       266
                Travel       0.70      0.83      0.76       282

              accuracy                           0.77      1399
             macro avg       0.79      0.77      0.78      1399
          weighted avg       0.78      0.77      0.77      1399



In [89]:
clf_=XGBClassifier(n_estimators=300,max_depth=7,learning_rate=0.02)
clf_.fit(train_mat,y_train)
preds=clf_.predict(valid_mat)

print (classification_report(y_true=y_valid,y_pred=preds))

                        precision    recall  f1-score   support

         Art and Music       0.85      0.67      0.75       231
                  Food       0.85      0.74      0.79       163
               History       0.81      0.61      0.70       205
         Manufacturing       0.93      0.66      0.77       252
Science and Technology       0.46      0.89      0.60       266
                Travel       0.83      0.66      0.74       282

              accuracy                           0.71      1399
             macro avg       0.79      0.71      0.72      1399
          weighted avg       0.78      0.71      0.72      1399



In [0]:

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers

In [0]:
train_mat=train_mat.toarray()[:,:,None]
valid_mat=valid_mat.toarray()[:,:,None]


In [0]:
import tensorflow.keras.backend as K

In [0]:
#need embedding tfidf doesnt work with keras
train_mat=K.variable(train_mat)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(train_mat)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=train_mat, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(train_mat, y_train, epochs=1, verbose=1)